In [145]:
import pandas as pd
import numpy as np

import statsmodels.api as sm

from train import return_vif, calculate_r_squared, scale_data

In [146]:
path = '~/Downloads/train_set.csv'
data = pd.read_csv(path, sep='\t')
data.columns

Index(['ID', 'Log price (1 billion VND)', 'Log area (square meters)',
       'Bedrooms', 'WC', 'Parking', 'For rent', 'Infrastructure', 'Facade',
       'Number of floors', 'Log usable area (square meters)', 'Latitude',
       'Longitude', 'Cluster', 'Distance to center 0', 'Distance to center 1',
       'Distance to center 2', 'Distance to center 3', 'Distance to center 4'],
      dtype='object')

In [147]:
data  = scale_data(data)

In [148]:
data["Price"] = np.exp(data["Log price (1 billion VND)"])
data["Area"] = np.exp(data["Log area (square meters)"])
data["Usable Area"] = np.exp(data["Log usable area (square meters)"])
data.drop(columns="Log price (1 billion VND)", inplace=True)
data.drop(columns="Log area (square meters)", inplace=True)
data.drop(columns="Log usable area (square meters)", inplace=True)
# data.drop(columns=["ID",
#                     "Log price (1 billion VND)", 
#                     "Parking",
#                     "Log area (square meters)", 
#                     'Log usable area (square meters)',
#                     "WC",
#                     "Latitude", 
#                     "Longitude",  
#                     "Number of floors", 
#                     "Cluster",
#                     'Distance to center 0', 
#                     'Distance to center 1', 
#                     'Distance to center 2', 
#                     'Distance to center 3', 
#                     'Distance to center 4'
#                    ], inplace=True)

In [149]:
data.head()

,ID,Bedrooms,WC,Parking,For rent,Infrastructure,Facade,Number of floors,Latitude,Longitude,Cluster,Distance to center 0,Distance to center 1,Distance to center 2,Distance to center 3,Distance to center 4,Price,Area,Usable Area
0,0.004437,-0.400485,0.082193,-0.887428,-0.321481,0.671414,-0.748655,0.932693,1.354486,-1.140584,1.328469,-1.322585,1.351765,0.172328,-1.380141,1.388260,0.588423,0.275586,0.821707
1,0.650695,-0.400485,-0.843826,-0.887428,-0.321481,-1.489394,-0.748655,-0.707129,0.317790,2.728588,0.328240,-0.371706,0.343096,-4.732391,-0.235886,-0.330580,4.566665,0.528803,0.370746
2,0.910272,0.030096,0.082193,1.126852,-0.321481,0.671414,1.335729,-0.707129,-0.765820,0.424308,-0.671988,0.764754,-0.784488,0.192896,0.762074,-0.764218,0.464030,1.391436,0.770017
3,-1.520802,-0.831066,-0.843826,-0.887428,-0.321481,-1.489394,1.335729,-0.816450,-0.778887,0.211759,-0.671988,0.778967,-0.798698,0.279746,0.772966,-0.718391,0.877389,0.259272,0.186595
4,-0.640030,-0.831066,-0.843826,1.126852,-0.321481,-1.489394,-0.748655,-0.707129,-0.787789,0.225768,-0.671988,0.788712,-0.799020,0.316293,0.782069,-0.713225,0.904976,0.696010,0.456267


In [150]:
X_train = data.drop("Price", axis=1)
y_train = data["Price"]

In [151]:
return_vif(X_train)

,Features,VIF
0,ID,1.01449
1,Bedrooms,3.27904
2,WC,3.35004
3,Parking,1.07006
4,For rent,1.04940
5,Infrastructure,1.01361
6,Facade,1.04815
7,Number of floors,2.51753
8,Latitude,1288.26336
9,Longitude,50.24168


In [152]:
X_train.corr()

,ID,Bedrooms,WC,Parking,For rent,Infrastructure,Facade,Number of floors,Latitude,Longitude,Cluster,Distance to center 0,Distance to center 1,Distance to center 2,Distance to center 3,Distance to center 4,Area,Usable Area
ID,1.000000e+00,-0.026392,-0.046307,-0.021767,0.013417,-0.004171,3.550333e-07,-0.058606,0.012167,0.021737,0.000541,-0.014407,0.013036,-0.058890,-0.012245,0.008072,-0.022206,-0.047003
Bedrooms,-2.639233e-02,1.000000,0.812025,0.045758,0.157073,0.010205,3.806968e-02,0.536207,0.073661,-0.040623,0.083992,-0.069454,0.072052,-0.030061,-0.072103,0.069213,0.221338,0.526053
WC,-4.630701e-02,0.812025,1.000000,0.083544,0.183690,0.011383,3.447687e-02,0.583467,0.109007,-0.075519,0.122247,-0.103004,0.107094,-0.027546,-0.107378,0.106813,0.113239,0.426213
Parking,-2.176733e-02,0.045758,0.083544,1.000000,-0.021804,-0.017113,-7.311108e-02,0.168508,0.110368,-0.168298,0.132505,-0.105283,0.109396,0.102392,-0.115329,0.129904,-0.031697,0.032620
For rent,1.341698e-02,0.157073,0.183690,-0.021804,1.000000,0.033798,2.735766e-02,0.121453,-0.011804,0.043172,-0.000923,0.012897,-0.012599,-0.064675,0.013826,-0.021361,0.087183,0.123752
Infrastructure,-4.171229e-03,0.010205,0.011383,-0.017113,0.033798,1.000000,-3.601202e-03,0.041549,0.058890,-0.054979,0.064438,-0.057618,0.060434,0.011402,-0.060868,0.062141,-0.006709,0.028767
Facade,3.550333e-07,0.038070,0.034477,-0.073111,0.027358,-0.003601,1.000000e+00,0.029087,0.114975,-0.056318,0.069307,-0.118155,0.116020,0.018637,-0.115383,0.113416,0.105620,0.038415
Number of floors,-5.860650e-02,0.536207,0.583467,0.168508,0.121453,0.041549,2.908694e-02,1.000000,0.319292,-0.244970,0.303523,-0.314533,0.315822,0.027280,-0.323835,0.323786,0.038149,0.550816
Latitude,1.216741e-02,0.073661,0.109007,0.110368,-0.011804,0.058890,1.149754e-01,0.319292,1.000000,-0.576903,0.781463,-0.998263,0.998587,-0.135805,-0.998773,0.981484,-0.156524,-0.037588
Longitude,2.173734e-02,-0.040623,-0.075519,-0.168298,0.043172,-0.054979,-5.631821e-02,-0.244970,-0.576903,1.000000,-0.411151,0.555225,-0.566050,-0.593665,0.590340,-0.715006,0.166201,0.044035


In [153]:
X_train = sm.add_constant(X_train)
ols = sm.OLS(y_train, X_train).fit()
print(ols.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.217
Model:                            OLS   Adj. R-squared:                  0.208
Method:                 Least Squares   F-statistic:                     23.54
Date:                Mon, 17 Jun 2024   Prob (F-statistic):           9.37e-69
Time:                        20:52:33   Log-Likelihood:                -3339.8
No. Observations:                1548   AIC:                             6718.
Df Residuals:                    1529   BIC:                             6819.
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    1.4081 

In [154]:
r_squared = pd.DataFrame()
r_squared["columns"] = X_train.columns
r_squared["r2"] = [calculate_r_squared(df=X_train, column=column) for column in X_train.columns]
r_squared

/home/tan/miniconda3/envs/handsonds/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1781: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


,columns,r2
0,const,-inf
1,ID,0.014387
2,Bedrooms,0.700427
3,WC,0.702628
4,Parking,0.065480
5,For rent,0.047797
6,Infrastructure,0.013438
7,Facade,0.049136
8,Number of floors,0.604536
9,Latitude,0.999224


In [155]:
ols.rsquared_adj

0.20778000028919175